### ЗАДАНИЕ:

1. Оценить количество ботов, которые заходят на наш сайт, и их долю от всего трафика.
2. Определить, с каких источников и в какое время суток чаще всего заходят роботы.

Импортируем модули, прописываем хэдер для авторизации.

In [610]:
import json 
import requests
from pprint import pprint
import pandas as pd

token = 'ЗДЕСЬ НУЖЕН ТОКЕН'
headers = {'Authorization': 'OAuth ' + token}

Формируем параметры запроса, производим и обрабатываем запрос.

In [611]:
params = {'metrics': 'ym:s:visits,ym:s:robotPercentage',
          'dimensions': 'ym:s:lastsignTrafficSource,ym:s:hour',  
          'date1': '2020-09-01',  
          'date2': '2020-09-30',     
          'ids': 30177909,
          'accuracy':'full',
          'limit':100000}
response = requests.get('https://api-metrika.yandex.net/stat/v1/data', params=params, headers=headers)
print (response.status_code)
metrika_data = response.json()
metrika_df = pd.DataFrame(metrika_data['data'])
print(metrika_df.head(10))
print(metrika_df.info())

200
                                          dimensions               metrics
0  [{'icon_id': '2', 'icon_type': 'traffic-source...  [251.0, 24.70119522]
1  [{'icon_id': '2', 'icon_type': 'traffic-source...        [240.0, 26.25]
2  [{'icon_id': '2', 'icon_type': 'traffic-source...  [228.0, 25.87719298]
3  [{'icon_id': '2', 'icon_type': 'traffic-source...   [227.0, 28.1938326]
4  [{'icon_id': '2', 'icon_type': 'traffic-source...  [225.0, 24.88888889]
5  [{'icon_id': '2', 'icon_type': 'traffic-source...  [225.0, 25.77777778]
6  [{'icon_id': '2', 'icon_type': 'traffic-source...  [209.0, 26.31578947]
7  [{'icon_id': '2', 'icon_type': 'traffic-source...  [186.0, 19.35483871]
8  [{'icon_id': '2', 'icon_type': 'traffic-source...  [166.0, 17.46987952]
9  [{'icon_id': '2', 'icon_type': 'traffic-source...  [161.0, 23.60248447]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107 entries, 0 to 106
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      ----------

Преобразуем данные, избавившись от вложенных словарей с помощью функции, и запишем их в новую переменную.

In [612]:
def getMetrikaDataInListOfDicts(metrika_data):
    list_of_dicts = []
    dimensions_list = metrika_data['query']['dimensions']
    metrics_list = metrika_data['query']['metrics']
    for data_item in metrika_data['data']:
        d = {}
        for i,dimension in enumerate(data_item['dimensions']):
            d[dimensions_list[i]] = dimension['name']
        for i,metric in enumerate(data_item['metrics']):
            d[metrics_list[i]] = metric
        list_of_dicts.append(d)
    return list_of_dicts

metrika_list_of_dicts = getMetrikaDataInListOfDicts(metrika_data)
pprint(metrika_list_of_dicts[0:5])

[{'ym:s:hour': '16:00',
  'ym:s:lastsignTrafficSource': 'Search engine traffic',
  'ym:s:robotPercentage': 24.70119522,
  'ym:s:visits': 251.0},
 {'ym:s:hour': '15:00',
  'ym:s:lastsignTrafficSource': 'Search engine traffic',
  'ym:s:robotPercentage': 26.25,
  'ym:s:visits': 240.0},
 {'ym:s:hour': '11:00',
  'ym:s:lastsignTrafficSource': 'Search engine traffic',
  'ym:s:robotPercentage': 25.87719298,
  'ym:s:visits': 228.0},
 {'ym:s:hour': '13:00',
  'ym:s:lastsignTrafficSource': 'Search engine traffic',
  'ym:s:robotPercentage': 28.1938326,
  'ym:s:visits': 227.0},
 {'ym:s:hour': '12:00',
  'ym:s:lastsignTrafficSource': 'Search engine traffic',
  'ym:s:robotPercentage': 24.88888889,
  'ym:s:visits': 225.0}]


Из преобразованных данных создаем датафрейм.

In [613]:
metrika_df = pd.DataFrame(metrika_list_of_dicts)
print(metrika_df.head(10))
print(metrika_df.info())

  ym:s:lastsignTrafficSource ym:s:hour  ym:s:visits  ym:s:robotPercentage
0      Search engine traffic     16:00        251.0             24.701195
1      Search engine traffic     15:00        240.0             26.250000
2      Search engine traffic     11:00        228.0             25.877193
3      Search engine traffic     13:00        227.0             28.193833
4      Search engine traffic     12:00        225.0             24.888889
5      Search engine traffic     14:00        225.0             25.777778
6      Search engine traffic     17:00        209.0             26.315789
7      Search engine traffic     10:00        186.0             19.354839
8      Search engine traffic     21:00        166.0             17.469880
9      Search engine traffic     19:00        161.0             23.602484
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107 entries, 0 to 106
Data columns (total 4 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------              

Переименуем столбцы. Оценим количество ботов, которые заходят на сайт.

In [614]:
metrika_df.columns=['source', 'hour', 'visits', 'percentage']
# сумма всех визитов
sum_of_visits = metrika_df['visits'].sum()
# добавим колонку количества визитов ботов
metrika_df['bot_visits'] = metrika_df['visits'] * metrika_df['percentage'] / 100
# сумма визитов ботов
sum_of_bot_visits = metrika_df['bot_visits'].sum()
# доля от всего трафика
percent_of_bot_visits = sum_of_bot_visits / sum_of_visits * 100
sum_of_bot_visits = round(sum_of_bot_visits)
percent_of_bot_visits = round(percent_of_bot_visits, 2)
print('Колличество визитов ботов в сентябре 2020 года: {}.'.format(sum_of_bot_visits))
print('Доля визитов ботов от общего трафика в сентябре 2020 года: {} процентов.'.format(percent_of_bot_visits))

Колличество визитов ботов в сентябре 2020 года: 962.
Доля визитов ботов от общего трафика в сентябре 2020 года: 22.29 процентов.


Определим, из каких источников и в какое время суток чаще всего заходят роботы.

In [615]:
# удаляем лишние столбцы
metrika_df2 = metrika_df.drop(['percentage', 'hour'], axis=1)
# группируем по источнику с агреггированием сумм по визитам, сортируем
metrika_df2 = metrika_df2.groupby(by='source')[['visits', 'bot_visits']].apply(sum)
metrika_df2 = metrika_df2.sort_values('bot_visits', ascending=False)
print('Количество визитов ботов по источникам:')
print(metrika_df2)

Количество визитов ботов по источникам:
                               visits  bot_visits
source                                           
Search engine traffic          3334.0       739.0
Direct traffic                  757.0       177.0
Link traffic                    126.0        31.0
Social network traffic           80.0        11.0
Internal traffic                 10.0         3.0
Cached page traffic               6.0         1.0
Recommendation system traffic     2.0         0.0


In [616]:
# приводим столбец к типу datatime
metrika_df['hour'] = pd.to_datetime(metrika_df['hour'])
# добавляем столбец с часом, применив функцию к уже имеющемуся
metrika_df['hour2'] = metrika_df['hour'].dt.hour
# задаем множества значений для определения признаков времени
night = {0, 1, 2, 3, 4, 5}
morning = {6, 7, 8, 9, 10, 11}
afternoon = {12, 13, 14, 15, 16, 17}
evening = {18, 19, 20, 21, 22, 23}
# добавляем столбец с принадлежностью к группе, применив функцию с признаком времени
metrika_df['group']=metrika_df.apply(
    lambda x: 'Ночь' if x['hour2'] in night else (
                'Утро' if x['hour2'] in morning else (
                'День' if x['hour2'] in afternoon else (
                'Вечер' if x['hour2'] in evening else 'Нет данных'))
            ), 
    axis=1)
# удаляем ненужные столбцы
metrika_df3 = metrika_df.drop(['source', 'percentage', 'hour', 'hour2'], axis=1)
# группируем по времени визита с агреггированием сумм по визитам, сортируем
metrika_df3 = metrika_df3.groupby(by='group')[['visits', 'bot_visits']].apply(sum)
metrika_df3 = metrika_df3.sort_values('bot_visits', ascending=False)

print('Количество визитов ботов по времени суток:')
print(metrika_df3)

Количество визитов ботов по времени суток:
       visits  bot_visits
group                    
День   1745.0       441.0
Вечер  1190.0       238.0
Утро   1022.0       228.0
Ночь    358.0        55.0


### ВЫВОД

Количество визитов ботов в сентябре 2020 года составляет 962 единицы (или 22.29% от общего трафика). Наибольшее количество визитов из источников 'Search engine traffic' и 'Direct traffic'. Больше всего визиты ботов осуществляются днем, в равной степени утром и вечером.